# **Processamento de Linguagem Natural [2025-Q3]**
Prof. Alexandre Donizeti Alves

### **PROJETO PRÁTICO** [LangChain + Grandes Modelos de Linguagem]


O **PROJETO PRÁTICO** deve ser feito utilizando o **Google Colab** com uma conta sua vinculada ao Gmail. O link do seu notebook armazenado no Google Drive e o link de um repositório no GitHub devem ser enviados usando o seguinte formulário:

> https://forms.gle/D4gLqP1iGgyn2hbH8


**IMPORTANTE**: A submissão deve ser feita até o dia **07/12 (domingo)** APENAS POR UM INTEGRANTE DA EQUIPE, até às 23h59. Por favor, lembre-se de dar permissão de ACESSO IRRESTRITO para o professor da disciplina.

### **EQUIPE**

---

**POR FAVOR, PREENCHER OS INTEGRANDES DA SUA EQUIPE:**


**Integrante 01:**

`Bruno Henrrico Ancelani RA: 11202020477`

**Integrante 02:**

`Gustavo Breno Nunes Godinho RA: 11202130028`


### **GRANDE MODELO DE LINGUAGEM (*Large Language Model - LLM*)**

---

Cada equipe deve selecionar um Grande Modelo de Linguagem (*Large Language Model - LMM*).



Por favor, informe os dados do LLM selecionada:

>


**LLM**: GPT-4o mini (Open AI)

>

**Link para a documentação oficial**: https://platform.openai.com/docs/overview



### **API (Opcional)**
---

Por favor, informe os dados da API selecionada:

**API**: Open AI API

**Site oficial**: https://platform.openai.com/docs/overview

**Link para a documentação oficial**: https://platform.openai.com/docs/api-reference/introduction






### **DESCRIÇÃO**
---

Implementar um `notebook` no `Google Colab` que faça uso do framework **`LangChain`** (obrigatório) e de um **LLM** aplicando, no mínimo, DUAS técnicas de PLN. As técnicas podem ser aplicada em qualquer córpus obtido a partir de uma **API** ou a partir de uma página Web.

O **LLM** e a **API** selecionados devem ser informados na seguinte planilha:

> https://docs.google.com/spreadsheets/d/1iIUZcwnywO7RuF6VEJ8Rx9NDT1cwteyvsnkhYr0NWtU/edit?usp=sharing

>
As seguintes técnicas de PLN podem ser usadas:

*   Correção Gramatical
*   Classificação de Textos
*   Análise de Sentimentos
*   Detecção de Emoções
*   Extração de Palavras-chave
*   Tradução de Textos
*   Sumarização de Textos
*   Similaridade de Textos
*   Reconhecimento de Entidades Nomeadas
*   Sistemas de Perguntas e Respostas
>

**IMPORTANTE:** É obrigatório usar o e-mail da UFABC.


### **CRITÉRIOS DE AVALIAÇÃO**
---


Serão considerados como critérios de avaliação os seguintes pontos:

* Uso do framework **`LangChain`**.

* Escolha e uso de um **LLM**.

* Escolha e uso de uma **API** ou **Página Web**.

* Projeto disponível no Github.

* Apresentação (5 a 10 minutos).

* Criatividade no uso do framework **`LangChain`** em conjunto com o **LLM** e a **API**.




**IMPORTANTE**: todo o código do notebook deve ser executado. Código sem execução não será considerado.

### **IMPLEMENTAÇÃO**
---

### Conectando o Google Drive ao Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Instalando bibliotecas

In [ ]:
!pip install -q -U langchain langchain-openai langchain-community openai

### Importando bibliotecas

In [ ]:
#imports
import langchain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.middleware import OpenAIModerationMiddleware
from langchain_core.output_parsers import StrOutputParser

import openai
from openai import OpenAI

from google.colab import userdata

from IPython.display import Markdown

import pandas as pd
import hashlib
import json
import time

### Lendo dados do arquivo csv para um data frame e acessando informações de um twitte.



In [ ]:
df = pd.read_csv('/content/drive/MyDrive/PLN_2025/df_tox.csv')
date = df.loc[0, 'created_at']
user = df.loc[0, 'screen_name']
message = df.loc[0, 'full_text']

### Configurando o modelo da OpenAI

In [ ]:
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

model_analy = ChatOpenAI(
    model="gpt-4o-mini",
    temperature= 0,
    api_key = OPENAI_API_KEY
)

parser = StrOutputParser()

### Criando um template de prompt para moderação de mensagens

In [ ]:
pront_moder_summary = ChatPromptTemplate.from_template('''
  Você irá agir como um moderador de um chat de rede social, o seu objetivo é atuar lendo mensagens enviadas e resumir o conteúdo
  delas para que posteriormente possa ser feita uma audição das razões pela qual a mensagem será bloqueada. Para isso você deve realizar
  uma análise produzindo a seguinte estrutura:

  Alvo da mensagem: (A quem a mensagem se refere, veja se alguma hashtag está presente ou algum arroba de usuário)
  Tom da mensagem: (Agressivo, triste, ansioso ou ameaçador)
  Palavras chave: (Principais termos pejorativos usados na mensagem)
  Usuário que enviou: {user}
  Data de envio: {date} (Deixe em um formato mais legível)
  Mensagem: {message}
''')

### Executando a cadeia de moderação e exibindo o resumo

In [ ]:
chain_moder_summary = pront_moder_summary | model_analy | parser

summary_message = chain_moder_summary.invoke({
  'message': message,
  'user': user,
  'date': date
})

display(Markdown(summary_message))

Alvo da mensagem: @galloradical, @Brunosoares1821, @solonsampaio, @marciofrancasp, @GuilhermeBoulos, @freiredanilo09 (candidatura do França)

Tom da mensagem: Agressivo

Palavras chave: "bosta", "rasga o cu", "cacete"

Usuário que enviou: raffaeljuri  
Data de envio: 26 de outubro de 2020, 21:25 (UTC)  
Mensagem: Sou cirista e não apoio essa candidatura bosta do França, agora rasga o cu aí a vontade cacete.

### Usando a API da OpenAI para moderação de mensagens

In [ ]:
client = OpenAI( api_key = OPENAI_API_KEY )

moderation = client.moderations.create(input=message)

moderation_json = json.loads(moderation.model_dump_json())

results = moderation_json['results'][0]  # pega o primeiro item da lista
moderation_results = {
    'flagged': results['flagged'],
    'categories': results['categories']
}

### Resultado da API de moderação

In [ ]:
moderation_results

{'flagged': True,
 'categories': {'harassment': True,
  'harassment_threatening': False,
  'hate': False,
  'hate_threatening': False,
  'illicit': False,
  'illicit_violent': False,
  'self_harm': False,
  'self_harm_instructions': False,
  'self_harm_intent': False,
  'sexual': False,
  'sexual_minors': False,
  'violence': False,
  'violence_graphic': False,
  'harassment/threatening': False,
  'hate/threatening': False,
  'illicit/violent': False,
  'self-harm/intent': False,
  'self-harm/instructions': False,
  'self-harm': False,
  'sexual/minors': False,
  'violence/graphic': False}}

 ### Definindo descrições das categorias de moderação

In [ ]:
categories_desc = '''
harassment:	Content that expresses, incites, or promotes harassing language towards any target.

harassment/threatening:	Harassment content that also includes violence or serious harm towards any target.

hate:	Content that expresses, incites, or promotes hate based on race, gender, ethnicity, religion, nationality, sexual orientation, disability status, or caste. Hateful content aimed at non-protected groups (e.g., chess players) is harassment.

hate/threatening:	Hateful content that also includes violence or serious harm towards the targeted group based on race, gender, ethnicity, religion, nationality, sexual orientation, disability status, or caste.

illicit:	Content that gives advice or instruction on how to commit illicit acts. A phrase like "how to shoplift" would fit this category.

illicit/violent:	The same types of content flagged by the illicit category, but also includes references to violence or procuring a weapon.

self-harm:	Content that promotes, encourages, or depicts acts of self-harm, such as suicide, cutting, and eating disorders.

self-harm/intent:	Content where the speaker expresses that they are engaging or intend to engage in acts of self-harm, such as suicide,
cutting, and eating disorders.

self-harm/instructions:	Content that encourages performing acts of self-harm, such as suicide, cutting, and eating disorders, or that gives
instructions or advice on how to commit such acts.

sexual:	Content meant to arouse sexual excitement, such as the description of sexual activity, or that promotes sexual services (excluding sex education and wellness).

sexual/minors:	Sexual content that includes an individual who is under 18 years old.

violence:	Content that depicts death, violence, or physical injury.

violence/graphic:	Content that depicts death, violence, or physical injury in graphic detail.
'''

### Criando um prompt para relatório de análise de moderação

In [ ]:
pront_moder_analysis = ChatPromptTemplate.from_template('''
  Você irá agir como um especialista moderador de um chat de rede social, uma mensagem potencialmente nociva será enviada para você, e seu
  objetivo é produzir um breve relatório de qual seria o motivo do bloqueio da mesma, produza um texto em português.

  Nesse relatório resuma a mensagem e diga explicitamente as infrações que foram encontradas. Para te auxiliar será enviada uma cópia da mensagem,
  um dicionário com as infrações detectadas, a definicição em inglês de cada infração, não manter o rótulo do termo no idioma original no relatório (que pode ser utilizada no corpo do relatório, mas deve ser traduzida) e
  um breve resumo com algumas informações sobre a mensagem. Use as palavra chave presentes no resumo.

  Em casos de mensagens com muitas infrações, recomendar o banimento do usuário. Caso contrário, recomendar o bloqueio da mensagem e advertência.


  O relatório deve ser escrito como Markdown

  Mensagem: {message}
  Alvo da mensagem:
  Tom da mensagem:
  Palavras chave:
  Usuário que enviou:
  Data de envio:


  Dicionário: {moderation_results}
  Descrição: {categories_desc}
  Resumo: {summary_message}
''')

### Executando a análise final de moderação e exibindo o relatório

In [ ]:
chain_moder_analysis = pront_moder_analysis | model_analy | parser

analysis_message = chain_moder_analysis.invoke({
    'message': message,
    'moderation_results': moderation_results,
    'categories_desc': categories_desc,
    'summary_message': summary_message
})

display(Markdown(analysis_message))

# Relatório de Moderação

**Mensagem:** Sou cirista e não apoio essa candidatura bosta do França, agora rasga o cu aí a vontade cacete.  
**Alvo da mensagem:** @galloradical, @Brunosoares1821, @solonsampaio, @marciofrancasp, @GuilhermeBoulos, @freiredanilo09 (candidatura do França)  
**Tom da mensagem:** Agressivo  
**Palavras chave:** "bosta", "rasga o cu", "cacete"  
**Usuário que enviou:** raffaeljuri  
**Data de envio:** 26 de outubro de 2020, 21:25 (UTC)  

## Resumo da Mensagem
A mensagem enviada pelo usuário apresenta um tom agressivo e desrespeitoso, direcionado a várias pessoas, incluindo figuras públicas. O autor expressa sua desaprovação em relação a uma candidatura, utilizando linguagem ofensiva e vulgar.

## Infrações Detectadas
A mensagem contém as seguintes infrações:

- **Assédio:** A mensagem expressa linguagem que incita ou promove assédio em relação aos alvos mencionados, utilizando termos depreciativos e ofensivos.

Dada a gravidade da infração e o tom agressivo da mensagem, recomenda-se o **bloqueio da mensagem e advertência ao usuário**. É importante monitorar o comportamento do usuário para evitar futuras infrações.